In [3]:
import pandas as pd

In [25]:
pop2009 = pd.read_csv("POP2009.csv")
pop2009.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   NAME             53 non-null     object
 1   popestimate2009  53 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 980.0+ bytes


In [30]:
columns_to_keep1 = ["name", "popestimate2009"]
pop2009 = pop2009[columns_to_keep1]

# Define a dictionary to map old column names to new ones
column_renamed = {
    'NAME': 'name',
    'POPESTIMATE2009 ' : 'popestimate2009',
}

# Rename the columns using the mapping
pop2009.rename(columns=column_renamed, inplace=True)

# Save the DataFrame with the new column names to a new CSV file
pop2009.to_csv('POP2009.csv', index=False)

pop2009.info()

KeyError: "['name'] not in index"

In [11]:
pop2020 = pd.read_csv("POP2010-2020.csv")
pop2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   SUMLEV             57 non-null     int64 
 1   REGION             57 non-null     object
 2   DIVISION           57 non-null     object
 3   STATE              57 non-null     int64 
 4   NAME               57 non-null     object
 5   CENSUS2010POP      57 non-null     int64 
 6   ESTIMATESBASE2010  57 non-null     int64 
 7   POPESTIMATE2010    57 non-null     int64 
 8   POPESTIMATE2011    57 non-null     int64 
 9   POPESTIMATE2012    57 non-null     int64 
 10  POPESTIMATE2013    57 non-null     int64 
 11  POPESTIMATE2014    57 non-null     int64 
 12  POPESTIMATE2015    57 non-null     int64 
 13  POPESTIMATE2016    57 non-null     int64 
 14  POPESTIMATE2017    57 non-null     int64 
 15  POPESTIMATE2018    57 non-null     int64 
 16  POPESTIMATE2019    57 non-null     int64 
 17 

In [12]:
# Assuming nuforc is a Pandas DataFrame
columns_to_delete = ["SUMLEV", "REGION", "DIVISION", "STATE", "CENSUS2010POP", "ESTIMATESBASE2010", "POPESTIMATE042020", "POPESTIMATE2020" ]
pop2020 = pop2020.drop(columns=columns_to_delete)
pop2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   NAME             57 non-null     object
 1   POPESTIMATE2010  57 non-null     int64 
 2   POPESTIMATE2011  57 non-null     int64 
 3   POPESTIMATE2012  57 non-null     int64 
 4   POPESTIMATE2013  57 non-null     int64 
 5   POPESTIMATE2014  57 non-null     int64 
 6   POPESTIMATE2015  57 non-null     int64 
 7   POPESTIMATE2016  57 non-null     int64 
 8   POPESTIMATE2017  57 non-null     int64 
 9   POPESTIMATE2018  57 non-null     int64 
 10  POPESTIMATE2019  57 non-null     int64 
dtypes: int64(10), object(1)
memory usage: 5.0+ KB


In [18]:
pop2022 = pd.read_csv("POP2020-2022.csv")
pop2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 45 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   SUMLEV                 66 non-null     int64  
 1   REGION                 66 non-null     object 
 2   DIVISION               66 non-null     object 
 3   STATE                  66 non-null     int64  
 4   NAME                   66 non-null     object 
 5   ESTIMATESBASE2020      66 non-null     int64  
 6   POPESTIMATE2020        66 non-null     int64  
 7   POPESTIMATE2021        66 non-null     int64  
 8   POPESTIMATE2022        66 non-null     int64  
 9   NPOPCHG_2020           66 non-null     int64  
 10  NPOPCHG_2021           66 non-null     int64  
 11  NPOPCHG_2022           66 non-null     int64  
 12  BIRTHS2020             66 non-null     int64  
 13  BIRTHS2021             66 non-null     int64  
 14  BIRTHS2022             66 non-null     int64  
 15  DEATHS20

In [19]:
# Too many columns to delete, choosing a function that keeps certain columns instead
columns_to_keep2 = ["NAME", "POPESTIMATE2020", "POPESTIMATE2021", "POPESTIMATE2022"]
pop2022 = pop2022[columns_to_keep2]
pop2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   NAME             66 non-null     object
 1   POPESTIMATE2020  66 non-null     int64 
 2   POPESTIMATE2021  66 non-null     int64 
 3   POPESTIMATE2022  66 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 2.2+ KB


In [ ]:
merged_pop = pd.merge(pop2009, pop2020, on="NAME")

In [ ]:

data_path = "state_counts.csv"
ufo_data_path = "../nuforc_reports.csv"
population_data_path = ""

df = pd.read_csv(data_path)
shape = gpd.read_file(shape_path)
udf = pd.read_csv(ufo_data_path)

#Remove unnesecary columns
udf = udf.drop(columns=["summary", "country", "date_time", "shape", "duration"])

# Aggregate data

state_counts = udf['state'].value_counts()
city_counts = udf['city'].value_counts()


state_counts_df = pd.DataFrame({'state': state_counts.index, 
                                'stateCount': state_counts.values, 
                                'city': city_counts.index,
                                'cityCount' : city_counts.values,
                                'latitude':udf['city_latitude'], 
                                'longtitude': udf['city_longtitude']})

state_counts_file = 'counts.csv'
state_counts_df.to_csv(state_counts_file, index=False)



# Mapping of state abbreviations to full names
state_abbreviations = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts',
    'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
    'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico',
    'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}


# Clean and transform the 'state' column
df['state'] = df['state'].str.upper().str.strip().map(state_abbreviations)
df.dropna(subset=['state'], inplace=True)  # Drop rows with NaN values

# List of American state names
american_states = set(state_abbreviations.values())

# Filter the DataFrame to include only American states
df = df[df['state'].isin(american_states)]

# Group by 'state' and sum the 'count' values
df = df.groupby('state', as_index=False)['count'].sum()


,NAME,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE042020,POPESTIMATE2020
0,United States,309327143,311583481,313877662,316059947,318386329,320738994,323071755,325122128,326838199,328329953,329398742,329484123
1,Northeast Region,55380764,55608318,55782661,55912775,56021339,56052790,56063777,56083383,56084543,56002934,55924275,55849869
2,Midwest Region,66975328,67164092,67348275,67576524,67765576,67885682,68018175,68160342,68263019,68340091,68357895,68316744
3,South Region,114869421,116019483,117264196,118397213,119666248,121049223,122419547,123611036,124649156,125686544,126494232,126662754
4,West Region,72101630,72791588,73482530,74173435,74933166,75751299,76570256,77267367,77841481,78300384,78622340,78654756
5,Alabama,4785514,4799642,4816632,4831586,4843737,4854803,4866824,4877989,4891628,4907965,4920706,4921532
6,Alaska,713982,722349,730810,737626,737075,738430,742575,740983,736624,733603,732074,731158
7,Arizona,6407342,6473416,6556344,6634690,6732873,6832810,6944767,7048088,7164228,7291843,7393900,7421401
8,Arkansas,2921998,2941038,2952876,2960459,2968759,2979732,2991815,3003855,3012161,3020985,3029672,3030522
9,California,37319550,37636311,37944551,38253768,38586706,38904296,39149186,39337785,39437463,39437610,39418894,39368078
